# Get Coordinate Data to the Neighborhood Data

In [1]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Get the data from wiki
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Parse the html data using BeautifulSoup
soup = BeautifulSoup(wiki_url, 'html.parser')

# Find table in the website containing the data we need
postal_table = soup.find('table', attrs = {'rules': 'all'})

# Create a dataframe
column = ['PostalCode', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(columns = column)

# Filter every td and insert to df_toronto

for td in postal_table.find_all('td'):
    if td.span.text != 'Not assigned':
        postal_code = td.find('b').text
        borough = td.find('span').text.split('(')[0]
        neighbor = td.find('span').text.split('(')[1].strip(')').replace(' /', ',').replace(')', ' ').strip(' ')
    
        df_toronto = df_toronto.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighbor}, ignore_index = True)
        
# Replace the borough names which is too long
df_toronto['Borough']=df_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [26]:
# Get the coordinate data from the csv
import wget
loc_url = 'https://cocl.us/Geospatial_data'
loc_file = wget.download(loc_url)
df_loc = pd.read_csv(loc_file)
df_loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [28]:
# Change the column name of df_loc
df_loc.columns = ['PostalCode', 'Latitude', 'Longtitude']

# Merge the 2 dataframe
toronto_postalcode = pd.merge(df_toronto, df_loc[['PostalCode', 'Latitude', 'Longtitude']], on = 'PostalCode')

In [31]:
toronto_postalcode

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
